# Probability of Rolling each 2-dice point among 4 dice

- What is the probability of rolling any particular point value (from 2 to 12) for
  any roll of 4 dice?
  
  
Perhaps the most straightforward way to answer these questions is by exhaustive simulation
of 4 dice rolls (which only require $6^4 = 1296$ rolls).

In [81]:
from itertools import product

In [82]:
dice = (1,2,3,4,5,6)
all_rolls = list(product(dice, dice, dice, dice))
len(all_rolls)

1296

In [83]:
def points(dice):
    """ Return set of point values that can be derived from a given (a pair of dice)
        in a roll of the dice.
    """
    result = set()
    num_dice = len(dice)
    for first in range(num_dice):
        for second in range(first+1, num_dice):
            result.add(dice[first] + dice[second])
            
    return result

In [84]:
all_points = [points(x) for x in all_rolls]

In [85]:
def prob_point(p):
    c = 0
    for points in all_points:
        if p in points:
            c += 1
    return c / len(all_points)

In [86]:
for p in range(2, 13):
    print(f"{p:2d}: {prob_point(p):.1%}")

 2: 13.2%
 3: 23.3%
 4: 35.6%
 5: 44.8%
 6: 56.1%
 7: 64.4%
 8: 56.1%
 9: 44.8%
10: 35.6%
11: 23.3%
12: 13.2%


# Markov Modeling

Alternately, we can calculate the probabilities more directly by modeling
dice rolls with Markov processes and assigning a state to the specific point
value to be rolled.

For example, to model rolling a $2$ from a pair of dice among a sequence of
single-dice rolls a simple model need only consider when a $1$ is rolled, vs.
all the other faces on a die.  There is one way to roll a $1$ and five ways
of rolling another number.

# Roll 2

Looking for (1, 1).

<img src="./images/roll-2.png" width="200">

The state transitions can be modeled by a 3x3 array where the state-to-state
probabilities for the transition from state $i$ to $j$ is $A_{ij}$.

In [87]:
from pandas import DataFrame as df

roll_2 = df(((5/6, 0, 0), (1/6, 5/6, 0), (0, 1/6, 1)))
roll_2

,0,1,2
0,0.833333,0.000000,0
1,0.166667,0.833333,0
2,0.000000,0.166667,1


Multiplying a vector indicating the initial state, will result in the
probabilities of each state after a single roll.

In [88]:
roll_2.dot((1, 0, 0))

0    0.833333
1    0.166667
2    0.000000
dtype: float64

In [89]:
def matrix_power(m, n):
    result = m
    for _ in range(n-1):
        result = result.dot(m)
    return result

In [90]:
matrix_power(roll_2, 4)

,0,1,2
0,0.482253,0.000000,0.0
1,0.385802,0.482253,0.0
2,0.131944,0.517747,1.0


The probability of being in each state after 4 rolls of a die.

In [91]:
matrix_power(roll_2, 4).dot((1, 0, 0))

0    0.482253
1    0.385802
2    0.131944
dtype: float64

So, the probability of rolling a $2$ with four dices is $0.131944$.

We can come up with similar models for each of the other points (3, 4, 5, 6,
and 7) and note that by symmetry the points, $N$, above 7 are identical to the
models for $14 - N$.

# Roll 3

Looking for (1, 2).

<img src="./images/roll-3.png" width="200">

In [96]:
roll_3 = df(((4/6, 0, 0), (2/6, 5/6, 0), (0, 1/6, 1)))
print(roll_3)
print(matrix_power(roll_3, 4).dot((1, 0, 0)))

          0         1  2
0  0.666667  0.000000  0
1  0.333333  0.833333  0
2  0.000000  0.166667  1
0    0.197531
1    0.569444
2    0.233025
dtype: float64


# Roll 4

Looking for (1, 3) or (2, 2).

<img src="./images/roll-4.png" width="300">

In [93]:
roll_4 = df((
    (3/6, 0, 0, 0, 0),
    (2/6, 4/6, 0, 0, 0),
    (1/6, 0, 3/6, 0, 0),
    (0, 1/6, 2/6, 4/6, 0),
    (0, 1/6, 1/6, 2/6, 1)))
print(roll_4)
print(matrix_power(roll_4, 4).dot((1, 0, 0, 0, 0)))

          0         1         2         3  4
0  0.500000  0.000000  0.000000  0.000000  0
1  0.333333  0.666667  0.000000  0.000000  0
2  0.166667  0.000000  0.500000  0.000000  0
3  0.000000  0.166667  0.333333  0.666667  0
4  0.000000  0.166667  0.166667  0.333333  1
0    0.062500
1    0.270062
2    0.083333
3    0.228395
4    0.355710
dtype: float64


# Roll 5

Looking for (1, 4) or (2, 3).

<img src="./images/roll-5.png" width="300">

In [94]:
roll_5 = df((
    (2/6, 0, 0, 0),
    (4/6, 3/6, 0, 0),
    (0, 2/6, 4/6, 0),
    (0, 1/6, 2/6, 1))
)
print(roll_5)
print(matrix_power(roll_5, 4).dot((1, 0, 0, 0)))

          0         1         2  3
0  0.333333  0.000000  0.000000  0
1  0.666667  0.500000  0.000000  0
2  0.000000  0.333333  0.666667  0
3  0.000000  0.166667  0.333333  1
0    0.012346
1    0.200617
2    0.339506
3    0.447531
dtype: float64


# Roll 6

Looking for (1, 5), (2, 4), or (3, 3).

To simplify the model, we assign the symbol $a$ for the first of 1, 5, 2, or 4
to be rolled.  Then $a'$ would represent $6 - a$; the value that would pair
with $a$ to make $6$.

Then, $b$ would represent whichever is the first of values from 1, 5, 2, or 4
that are not $a$ or $a'$.  For example, if $a = 1$, then $b$ would be one of
2 or 4.

<img src="./images/roll-6.png" width="400">

In [102]:
roll_6 = 1 / 6 * df((
    (1, 0, 0, 0, 0, 0, 0),
    (4, 2, 0, 0, 0, 0, 0),
    (1, 0, 1, 0, 0, 0, 0),
    (0, 2, 0, 3, 0, 0, 0),
    (0, 1, 4, 0, 2, 0, 0),
    (0, 0, 0, 1, 2, 3, 0),
    (0, 1, 1, 2, 2, 3, 6)
))
print(roll_6)
print(matrix_power(roll_6, 4).dot((1, 0, 0, 0, 0, 0, 0)))

          0         1         2         3         4    5    6
0  0.166667  0.000000  0.000000  0.000000  0.000000  0.0  0.0
1  0.666667  0.333333  0.000000  0.000000  0.000000  0.0  0.0
2  0.166667  0.000000  0.166667  0.000000  0.000000  0.0  0.0
3  0.000000  0.333333  0.000000  0.500000  0.000000  0.0  0.0
4  0.000000  0.166667  0.666667  0.000000  0.333333  0.0  0.0
5  0.000000  0.000000  0.000000  0.166667  0.333333  0.5  0.0
6  0.000000  0.166667  0.166667  0.333333  0.333333  0.5  1.0
0    0.000772
1    0.046296
2    0.003086
3    0.154321
4    0.086420
5    0.148148
6    0.560957
dtype: float64


# Roll 7

Looking for (1, 6), (2, 5), or (3, 4).

To simplify the model, we assign the symbol $a$ for the first value rolled and
then $a'$ would be it's paired value, $7 - a$.

Similarly, the next two values from each of the subsequent pairs would be
assigned to $b$ and $c$.

As an example, if the sequence of 4 dice are (5, 6, 3, 1), we assign
$a = 5$, $b = 6$, and $c = 3$".

<img src="./images/roll-7.png" width="300">

In [104]:
roll_7 = 1 / 6 * df((
    (0, 0, 0, 0, 0),
    (6, 1, 0, 0, 0),
    (0, 4, 2, 0, 0),
    (0, 0, 2, 3, 0),
    (0, 1, 2, 3, 6),
))
print(roll_7)
print(matrix_power(roll_7, 4).dot((1, 0, 0, 0, 0)))

     0         1         2    3    4
0  0.0  0.000000  0.000000  0.0  0.0
1  1.0  0.166667  0.000000  0.0  0.0
2  0.0  0.666667  0.333333  0.0  0.0
3  0.0  0.000000  0.333333  0.5  0.0
4  0.0  0.166667  0.333333  0.5  1.0
0    0.000000
1    0.004630
2    0.129630
3    0.222222
4    0.643519
dtype: float64
